<a href="https://colab.research.google.com/github/Ruchir1218/Portfolio_Projects/blob/main/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pyspark questions practice

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from pyspark.sql import SparkSession
import os


In [ ]:
# Read a file and count word frequency in that file

spark = SparkSession.builder.appName("ReadPDF").getOrCreate()
pdf_directory = "/content/Hamlet.pdf"

# Function to extract text from a single PDF file
def extract_text_from_pdf(file_path):
    text = ""
    try:
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
    return text
pdf_text = extract_text_from_pdf(pdf_directory)
# print(type(pdf_text))


rdd = spark.sparkContext.parallelize([pdf_text])
# print(rdd.collect())

# this creates a rdd with each word seperated as a different string
words = rdd.flatMap(lambda x: x.split(" "))

# this gives value of 1 to every word
word_pair = words.map(lambda word:(word,1))

# reduce by key is used to collect similar words in same tuple and add their 1s
word_count = word_pair.reduceByKey(lambda x,y: x+y)

# sort the word count by descending order
sorted_word_count = word_count.sortBy(lambda x: x[1], ascending=False)

for word, count in sorted_word_count.collect():
    print(f"{word}: {count}")

spark.stop()

: 1372
the: 977
and: 693
of: 668
to: 649
I: 518
a: 470
my: 443
in: 403
you: 364
is: 318
his: 289
it: 267
not: 258
And: 253
that: 237
with: 234
your: 221
this: 208
be: 197
for: 178
have: 162
he: 159
as: 158
The: 154
me: 141
will: 133
but: 132
SC.: 132
To: 128
him: 126
are: 124
That: 122
But: 118
do: 117
our: 110
on: 110
shall: 107
we: 104
what: 103
by: 99
so: 99
from: 91
no: 87
What: 86
thou: 86
lord,: 85
all: 85
or: 84
thy: 82
was: 74
more: 73
her: 73
My: 73
at: 72
good: 71
For: 70
like: 69
they: 69
most: 69
As: 69
It: 67
you,: 65
A: 64
would: 63
know: 61
You: 59
may: 59
lord.: 59
th’: 59
very: 58
He: 58
their: 56
an: 56
O,: 56
hath: 56
If: 55
I’ll: 55
How: 54
if: 54
must: 54
some: 52
am: 51
should: 50
let: 50
them: 49
This: 49
us: 48
come: 47
O: 46
such: 46
did: 46
make: 46
Hamlet: 46
much: 44
With: 43
you.: 43
Let: 42
love: 42
which: 42
Of: 42
out: 42
now: 41
So: 41
In: 40
think: 39
upon: 39
own: 37
these: 37
than: 37
give: 37
had: 37
there: 37
she: 37
’tis: 37
speak: 36
sir,: 36
tel

Word frequency counter for text file

In [5]:
import pyspark
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("word_frequency").getOrCreate()
file_directory = "/content/words.txt"

words = spark.read.text(file_directory)

words_df = words.select(fn.explode(fn.split(fn.lower(fn.col("value")), " ")).alias("word"))

word_counts = words_df.groupBy("word").count().orderBy("count", ascending=False)
word_counts.show()

spark.stop()

+---------------+-----+
|           word|count|
+---------------+-----+
|               |   20|
|         python|   13|
|            sql|   12|
|        pyspark|   10|
|dataengineering|    8|
|            etl|    8|
|          spark|    7|
|     databricks|    6|
|datawarehousing|    6|
|azuredatabricks|    6|
|machinelearning|    5|
|          scala|    4|
|   dataanalysis|    4|
|        bigdata|    4|
|         hadoop|    4|
|           hive|    4|
|  azuredatalake|    3|
| microsoftazure|    3|
|      sqlserver|    3|
|      snowflake|    3|
+---------------+-----+
only showing top 20 rows

